## Hotmart Technical Case

- **Full Name:** Sarah Kelly Silva do Nascimento
- **Applied Position:** [Analista de Dados Pleno](https://hotmart.com/pt-br/trabalhe-conosco/vagas/4352224101)
- **Email:** [sarah.sqn@gmail.com](mailto:sarah.sqn@gmail.com)
- **LinkedIn:** [Sarah Kelly](https://www.linkedin.com/in/sarah-kelly-024351155/)

**The following scripts were used to create tables, insert data, and present information within this notebook. They do not need to be executed as the database was created locally on my machine. These scripts are provided for the evaluators' reference in the technical case study.**

In [11]:
# Connection Test

import pyodbc

server = 'SANDI-NOTEBOOK\\SKPROD01'
database = 'HotmartTechnicalCase'

try:
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
    print("Connection successful!")
    conn.close()
except Exception as e:
    print("Error connecting to SQL Server:", e)


Connection successful!


In [12]:
# Table Creation and Data Insertion

import pyodbc
import pandas as pd
import os

server = 'SANDI-NOTEBOOK\\SKPROD01'
database = 'HotmartTechnicalCase'

conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

csv_dir = r'C:\Users\sarah\Desktop\Technical Case\SQL Datasets'

csv_files = ['Producers.csv', 'Products.csv', 'Sales.csv']  

def create_table_and_insert_data(csv_file):
    csv_path = os.path.join(csv_dir, csv_file)
    df = pd.read_csv(csv_path)
  
    table_name = os.path.splitext(csv_file)[0]
    
    create_table_sql = f"CREATE TABLE {table_name} ("
    for col_name, col_type in zip(df.columns, df.dtypes):
        if col_type == 'int64':
            create_table_sql += f"{col_name} INT, "
        elif col_type == 'float64':
            create_table_sql += f"{col_name} FLOAT, "
        else:
            create_table_sql += f"{col_name} NVARCHAR(MAX), "
    create_table_sql = create_table_sql.rstrip(', ') + ")"
    
    cursor = conn.cursor()
    cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}")
    cursor.execute(create_table_sql)
    conn.commit()
    
    for index, row in df.iterrows():
        insert_sql = f"INSERT INTO {table_name} VALUES ("
        for value in row:
            if pd.isnull(value):
                insert_sql += "NULL, "
            else:
                insert_sql += f"'{value}', "
        insert_sql = insert_sql.rstrip(', ') + ")"
        cursor.execute(insert_sql)
    conn.commit()

for csv_file in csv_files:
    create_table_and_insert_data(csv_file)

conn.close()

print("Tables created and data inserted successfully in SQL Server.")


Tables created and data inserted successfully in SQL Server.


In [13]:
# Query Creation and Conversion to Variables

import pyodbc
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

server = 'SANDI-NOTEBOOK\\SKPROD01'
database = 'HotmartTechnicalCase'

conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

query1 = """
WITH ValidSales AS 
   (SELECT
        p.product_id,
        p.niche,
        ROW_NUMBER() OVER (PARTITION BY p.niche ORDER BY COUNT(s.purchase_id) DESC) AS rank
    FROM Products p
    LEFT JOIN Sales s ON p.product_id = s.product_id
        AND s.refund = 0
        AND s.cancelled = 0
        AND s.chargeback = 0
    WHERE p.member_area_active = 0
        AND p.recovery_active = 1
    GROUP BY
        p.product_id, 
        p.niche)
SELECT
    product_id,
    niche
FROM ValidSales
WHERE rank = 1
"""

Q1 = pd.read_sql(query1, conn)

query2 = """
WITH RankedProducers AS 
   (SELECT
        p.producer_id,
        ROW_NUMBER() OVER (ORDER BY MAX(s.comission_value) DESC) AS Rank
    FROM Producers p
    LEFT JOIN Products pr ON p.producer_id = pr.producer_id
    LEFT JOIN Sales s ON pr.product_id = s.product_id
    WHERE 1=1
        AND YEAR(p.registry_date) >= 2019 
        AND pr.recovery_active = 1
    GROUP BY
        p.producer_id)
SELECT
    rp.producer_id
FROM RankedProducers rp
LEFT JOIN Producers p ON rp.producer_id = p.producer_id
WHERE
    Rank <= 5
"""

Q2 = pd.read_sql(query2, conn)

query3 = """
WITH SalesWeek AS 
   (SELECT
        DATEPART(WEEKDAY, CONVERT(DATE, s.purchase_date)) AS WeekDay,
        DATEPART(YEAR, CONVERT(DATE, s.purchase_date)) AS Year,
        COUNT(*) AS total_sales
    FROM Sales s
    INNER JOIN Products pr ON s.product_id = pr.product_id
    INNER JOIN Producers p ON pr.producer_id = p.producer_id
    WHERE 1=1
        AND p.country = 'BRASIL'
        AND YEAR(s.purchase_date) IN (2020, 2021, 2022)
    GROUP BY
        DATEPART(YEAR, CONVERT(DATE, s.purchase_date)),
        DATEPART(WEEKDAY, CONVERT(DATE, s.purchase_date)))
SELECT
    CASE WeekDay
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
    END AS WeekDay,
    SUM(CASE WHEN Year = 2020 THEN total_sales ELSE 0 END) AS S_2020,
    SUM(CASE WHEN Year = 2021 THEN total_sales ELSE 0 END) AS S_2021,
    SUM(CASE WHEN Year = 2022 THEN total_sales ELSE 0 END) AS S_2022,
    SUM(CASE WHEN Year = 2021 THEN total_sales ELSE 0 END) - SUM(CASE WHEN Year = 2020 THEN total_sales ELSE 0 END) AS V_2021_2020,
    SUM(CASE WHEN Year = 2022 THEN total_sales ELSE 0 END) - SUM(CASE WHEN Year = 2021 THEN total_sales ELSE 0 END) AS V_2022_2021
FROM SalesWeek
GROUP BY
    WeekDay
ORDER BY
    WeekDay
"""

Q3 = pd.read_sql(query3, conn)

query4 = """
WITH SalesSummary AS 
   (SELECT
        p.country AS country,
		p.producer_id,
        COUNT(*) AS total_sales,
        SUM(s.cancelled) AS total_cancellations,
        SUM(s.refund) AS total_refunds
    FROM Sales s
    INNER JOIN Products pr ON s.product_id = pr.product_id
    INNER JOIN Producers p ON pr.producer_id = p.producer_id
    WHERE
        pr.type = 'Assinatura'
    GROUP BY
        p.country,
		p.producer_id)
SELECT
    country,
	producer_id,
    total_sales,
    total_cancellations,
    total_refunds,    
	FORMAT((total_cancellations + total_refunds) * 1.0 / total_sales * 100.0, 'N2') + '%' AS loss_performance
FROM SalesSummary
ORDER BY
	country,
    (total_cancellations + total_refunds) * 1.0 / total_sales * 100.0 DESC
"""

Q4 = pd.read_sql(query4, conn)

query5 = """
WITH CommissionTotals AS (
    SELECT
        s.product_id,
        SUM(CAST(REPLACE(s.comission_value, ',', '') AS FLOAT)) AS total_commission
    FROM Sales s
    WHERE
        s.cancelled = 1
    GROUP BY
        s.product_id
)
SELECT
    CASE
        WHEN p.recovery_active = 1 THEN 'RecoveryTrue'
        ELSE 'RecoveryFalse'
    END AS recovery_status,
    p.product_id,
    FORMAT(AVG(CAST(REPLACE(s.comission_value, ',', '') AS FLOAT) / ct.total_commission) * 100, 'N2')+'%' AS commission_retention
FROM Products p
    INNER JOIN Sales s ON p.product_id = s.product_id
    INNER JOIN CommissionTotals ct ON s.product_id = ct.product_id
WHERE 1=1
    AND p.registry_date > '2019-01-01'
    AND p.type = 'Curso'
    AND s.cancelled = 1
GROUP BY	
    CASE
        WHEN p.recovery_active = 1 THEN 'RecoveryTrue'
        ELSE 'RecoveryFalse'
    END,
    p.product_id
"""

Q5 = pd.read_sql(query5, conn)

query6 = """
WITH Rankings AS (
    SELECT
        p.producer_id,
        COUNT(s.purchase_id) AS total_sales,
        SUM(CAST(s.product_price AS FLOAT) * (1 - COALESCE(s.discount, 0) / 100) * (1 + COALESCE(s.service_tax, 0) / 100)) AS total_revenue,
        AVG(CAST(s.cancelled AS FLOAT)) AS cancellation_rate,
        SUM(CAST(pr.recovery_active AS FLOAT)) AS total_recovery_active,
        SUM(CAST(s.has_coupon AS FLOAT)) / COUNT(s.purchase_id) AS coupon_usage_rate,
        SUM(CAST(s.chargeback AS FLOAT)) / COUNT(s.purchase_id) AS chargeback_rate
    FROM Producers p
    INNER JOIN Products pr ON p.producer_id = pr.producer_id
    INNER JOIN Sales s ON 1=1
		AND pr.product_id = s.product_id
		AND YEAR(s.purchase_date) = 2023
    GROUP BY
        p.producer_id
)
SELECT
    r.producer_id,
    r.total_sales,
    r.total_revenue,
    FORMAT(r.cancellation_rate * 100, 'N1')+'%' AS cancellation_rate,
    FORMAT(r.coupon_usage_rate * 100, 'N1')+'%' AS coupon_usage_rate,
    FORMAT(r.chargeback_rate * 100, 'N1')+'%' AS chargeback_rate,
    r.total_recovery_active,
    RANK() OVER (ORDER BY r.total_sales DESC) AS sales_rank,
    RANK() OVER (ORDER BY r.total_revenue DESC) AS revenue_rank,
    RANK() OVER (ORDER BY r.cancellation_rate ASC) AS cancellation_rank
FROM Rankings r
ORDER BY
    r.total_sales DESC
"""

Q6 = pd.read_sql(query6, conn)

conn.close()


### **1. The top product with most valid purchases (without refund, cancellation, or chargeback), in each niche with deactivated membership area and activated recovery.**


 **SQL Query Used:**
   ```sql
WITH ValidSales AS 
   (SELECT
        p.product_id,
        p.niche,
        ROW_NUMBER() OVER (PARTITION BY p.niche ORDER BY COUNT(s.purchase_id) DESC) AS rank
    FROM Products p
    LEFT JOIN Sales s ON 1=1
		    AND p.product_id = s.product_id
        AND s.refund = 0
        AND s.cancelled = 0
        AND s.chargeback = 0
    WHERE 1=1
        AND p.member_area_active = 0
        AND p.recovery_active = 1
    GROUP BY
        p.product_id, 
		    p.niche)
SELECT
    product_id,
    niche
FROM ValidSales
WHERE
    rank = 1
  ```

In [14]:
Q1.head(100)

,product_id,niche
0,273593751,Artes e Design
1,983844542,Empreendedorismo
2,348532296,Financas
3,428909199,Marketing
4,260868848,Saúde e Alimentação
5,719732475,Tecnologia e Inovação
6,324917616,Viagens


### **2. The top 5 producers who joined Hotmart from 2019 onwards and achieved the highest commission using recovery.**

**SQL Query Used:**
   ```sql
WITH RankedProducers AS 
   (SELECT
        p.producer_id,
        ROW_NUMBER() OVER (ORDER BY MAX(s.comission_value) DESC) AS Rank
    FROM Producers p
    LEFT JOIN Products pr ON p.producer_id = pr.producer_id
    LEFT JOIN Sales s ON pr.product_id = s.product_id
    WHERE 1=1
        AND YEAR(p.registry_date) >= 2019 
        AND pr.recovery_active = 1
    GROUP BY
        p.producer_id)
SELECT
    rp.producer_id
FROM RankedProducers rp
LEFT JOIN Producers p ON rp.producer_id = p.producer_id
WHERE
    Rank <= 5
  ```

In [15]:
Q2.head(100)

,producer_id
0,6
1,13
2,5
3,10
4,3


### **3. List the number of completed sales from Brazilian producers in each weekday for the years 2020, 2021, and 2022 and the variation between the weekday sales from 2021/2020 and 2022/2021.**

 **SQL Query Used:**
   ```sql
WITH SalesWeek AS 
   (SELECT
        DATEPART(WEEKDAY, CONVERT(DATE, s.purchase_date)) AS WeekDay,
        DATEPART(YEAR, CONVERT(DATE, s.purchase_date)) AS Year,
        COUNT(*) AS total_sales
    FROM Sales s
    INNER JOIN Products pr ON s.product_id = pr.product_id
    INNER JOIN Producers p ON pr.producer_id = p.producer_id
    WHERE 1=1
        AND p.country = 'BRASIL'
        AND YEAR(s.purchase_date) IN (2020, 2021, 2022)
    GROUP BY
        DATEPART(YEAR, CONVERT(DATE, s.purchase_date)),
        DATEPART(WEEKDAY, CONVERT(DATE, s.purchase_date)))
SELECT
    CASE WeekDay
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
    END AS WeekDay,
    SUM(CASE WHEN Year = 2020 THEN total_sales ELSE 0 END) AS S_2020,
    SUM(CASE WHEN Year = 2021 THEN total_sales ELSE 0 END) AS S_2021,
    SUM(CASE WHEN Year = 2022 THEN total_sales ELSE 0 END) AS S_2022,
    SUM(CASE WHEN Year = 2021 THEN total_sales ELSE 0 END) - SUM(CASE WHEN Year = 2020 THEN total_sales ELSE 0 END) AS V_2021_2020,
    SUM(CASE WHEN Year = 2022 THEN total_sales ELSE 0 END) - SUM(CASE WHEN Year = 2021 THEN total_sales ELSE 0 END) AS V_2022_2021
FROM SalesWeek
GROUP BY
    WeekDay
ORDER BY
    WeekDay

  ```

In [16]:
Q3.head(100)

,WeekDay,S_2020,S_2021,S_2022,V_2021_2020,V_2022_2021
0,Friday,332,342,484,10,142
1,Monday,272,291,398,19,107
2,Saturday,504,541,746,37,205
3,Sunday,440,476,549,36,73
4,Thursday,215,194,327,-21,133
5,Tuesday,258,267,438,9,171
6,Wednesday,199,203,289,4,86


### **4. List each countrys loss performance for each producer country, considering only products from type "Assinatura". loss_performance = (cancelations + refunds) / total_sales**

 **SQL Query Used:**
   ```sql
WITH SalesSummary AS 
   (SELECT
        p.country AS country,
		p.producer_id,
        COUNT(*) AS total_sales,
        SUM(s.cancelled) AS total_cancellations,
        SUM(s.refund) AS total_refunds
    FROM Sales s
    INNER JOIN Products pr ON s.product_id = pr.product_id
    INNER JOIN Producers p ON pr.producer_id = p.producer_id
    WHERE
        pr.type = 'Assinatura'
    GROUP BY
        p.country,
		p.producer_id)
SELECT
    country,
	producer_id,
    total_sales,
    total_cancellations,
    total_refunds,    
	FORMAT((total_cancellations + total_refunds) * 1.0 / total_sales * 100.0, 'N2') + '%' AS loss_performance
FROM SalesSummary
ORDER BY
	country,
    (total_cancellations + total_refunds) * 1.0 / total_sales * 100.0 DESC
    
  ```

In [17]:
Q4.head(100)

,country,producer_id,total_sales,total_cancellations,total_refunds,loss_performance
0,BRASIL,7,195,36,9,"23,08%"
1,BRASIL,3,451,70,26,"21,29%"
2,BRASIL,8,168,19,15,"20,24%"
3,BRASIL,13,760,44,47,"11,97%"
4,BRASIL,1,1235,54,57,"8,99%"
5,COLOMBIA,4,546,77,48,"22,89%"
6,COLOMBIA,2,177,9,11,"11,30%"
7,EUA,9,72,6,3,"12,50%"
8,MEXICO,6,220,6,14,"9,09%"


### **5. Looking at the products registered after 01/2019, with at least one cancellation and from the product type ‘Curso’, calculate the average retention performance for commission, of all products, with recovery active and without recovery active. Is there any difference for product performance considering products with the recovery tool activated? average_commission_retention = commission_received/total_comission**

 **Response:**

Based on the data, it appears there are limited observations to draw definitive conclusions, particularly because there is only one product that meets the criteria with recovery not activated. However, for products with recovery activated, we observe a wide range in commission retention rates, varying from 2,63% to 50%.

 **SQL Query Used:**
   ```sql
WITH CommissionTotals AS (
    SELECT
        s.product_id,
        SUM(CAST(REPLACE(s.comission_value, ',', '') AS FLOAT)) AS total_commission
    FROM Sales s
    WHERE
        s.cancelled = 1
    GROUP BY
        s.product_id
)
SELECT
    CASE
        WHEN p.recovery_active = 1 THEN 'RecoveryTrue'
        ELSE 'RecoveryFalse'
    END AS recovery_status,
    p.product_id,
    FORMAT(AVG(CAST(REPLACE(s.comission_value, ',', '') AS FLOAT) / ct.total_commission) * 100, 'N2')+'%' AS commission_retention
FROM Products p
    INNER JOIN Sales s ON p.product_id = s.product_id
    INNER JOIN CommissionTotals ct ON s.product_id = ct.product_id
WHERE 1=1
    AND p.registry_date > '2019-01-01'
    AND p.type = 'Curso'
    AND s.cancelled = 1
GROUP BY	
    CASE
        WHEN p.recovery_active = 1 THEN 'RecoveryTrue'
        ELSE 'RecoveryFalse'
    END,
    p.product_id
    
  ```

In [18]:
Q5.head(100)

,recovery_status,product_id,commission_retention
0,RecoveryTrue,218834610,"9,09%"
1,RecoveryTrue,844949729,"5,56%"
2,RecoveryTrue,719732475,"2,63%"
3,RecoveryTrue,328762963,"16,67%"
4,RecoveryTrue,466262361,"33,33%"
5,RecoveryTrue,903828613,"50,00%"
6,RecoveryTrue,264261718,"20,00%"
7,RecoveryFalse,545252229,"4,17%"
8,RecoveryTrue,348532296,"11,11%"


### **6. If you need to create a ranking of the top creators of 2023, which variables you consider crucial for ranking them? You can also create variables from the data. You must explain your reasoning and your choice of variables and show how this reflect in your SQL code.**

 **Response:**

##### Common Table Expression (CTE) - Rankings

The query begins with a CTE named Rankings, which calculates metrics for each producer (`producer_id`). The following fields are selected:

- `producer_id`: Unique identifier of the producer.
- `total_sales`: Total count of sales made by the producer in the year 2023.
- `total_revenue`: Sum of total revenue generated by sales of the producer's products in 2023. This is calculated by multiplying the product price by the discount and service tax (if any).
- `cancellation_rate`: Average cancellation rate of sales made by the producer. It calculates the average of the `cancelled` column from the Sales table, indicating whether the sale was cancelled.
- `total_recovery_active`: Sum of recovery activation for the producer's products. This column comes from the Products table and indicates if sales recovery is active for each product.
- `coupon_usage_rate`: Rate of coupon usage in sales made by the producer. It is calculated by dividing the sum of the `has_coupon` column from the Sales table by the total number of sales (`COUNT(s.purchase_id)`).
- `chargeback_rate`: Rate of chargebacks in sales made by the producer. It is calculated by dividing the sum of the `chargeback` column from the Sales table by the total number of sales (`COUNT(s.purchase_id)`).

##### Final Selection

In the final selection (`SELECT`), fields chosen to display in the final result include:

- `producer_id`: Unique identifier of the producer.
- `total_sales`: Total count of sales.
- `total_revenue`: Total revenue generated.
- `cancellation_rate`: Cancellation rate formatted to display as a percentage.
- `coupon_usage_rate`: Coupon usage rate formatted to display as a percentage.
- `chargeback_rate`: Chargeback rate formatted to display as a percentage.
- `total_recovery_active`: Total activation of recovery.
- `sales_rank`: Ranking of the producer based on total sales, ordered from highest to lowest (`DESC`).
- `revenue_rank`: Ranking of the producer based on total revenue generated, ordered from highest to lowest (`DESC`).
- `cancellation_rank`: Ranking of the producer based on cancellation rate, ordered from lowest to highest (`ASC`).

##### Ordering

Results are ordered first by `total_sales` in descending order to show producers with the most sales at the top.

##### Explanation of Indicators

- **Total Sales**: Indicates the total number of sales made by each producer in the year 2023.
- **Total Revenue**: Shows the total revenue generated by sales of each producer's products in 2023, considering discounts and service fees.
- **Cancellation Rate**: Average rate of sales cancellations for each producer. A lower rate indicates better customer retention.
- **Coupon Usage Rate**: Percentage of sales that used discount coupons. Indicates the impact of coupons on the producer's sales strategy.
- **Chargeback Rate**: Percentage of sales that resulted in chargebacks. A high rate may indicate issues with payment disputes.
- **Total Recovery Active**: Sum of recovery activation for the producer's products, showing the use of this strategy to recover sales.

##### Percentage Formatting

To display cancellation rate, coupon usage rate, and chargeback rate as percentages, the `FORMAT()` function is used to format the values multiplied by 100, with one decimal place (`'N1'`), and concatenated with the percentage symbol (`%`).

##### Overview

This query provides a detailed overview of each producer's performance based on key metrics, facilitating comparison and ranking according to their results in 2023.


-------------------


 **SQL Query Used:**
   ```sql
WITH Rankings AS (
    SELECT
        p.producer_id,
        COUNT(s.purchase_id) AS total_sales,
        SUM(CAST(s.product_price AS FLOAT) * (1 - COALESCE(s.discount, 0) / 100) * (1 + COALESCE(s.service_tax, 0) / 100)) AS total_revenue,
        AVG(CAST(s.cancelled AS FLOAT)) AS cancellation_rate,
        SUM(CAST(pr.recovery_active AS FLOAT)) AS total_recovery_active,
        SUM(CAST(s.has_coupon AS FLOAT)) / COUNT(s.purchase_id) AS coupon_usage_rate,
        SUM(CAST(s.chargeback AS FLOAT)) / COUNT(s.purchase_id) AS chargeback_rate
    FROM Producers p
    INNER JOIN Products pr ON p.producer_id = pr.producer_id
    INNER JOIN Sales s ON 1=1
		AND pr.product_id = s.product_id
		AND YEAR(s.purchase_date) = 2023
    GROUP BY
        p.producer_id
)
SELECT
    r.producer_id,
    r.total_sales,
    r.total_revenue,
    FORMAT(r.cancellation_rate * 100, 'N1')+'%' AS cancellation_rate,
    FORMAT(r.coupon_usage_rate * 100, 'N1')+'%' AS coupon_usage_rate,
    FORMAT(r.chargeback_rate * 100, 'N1')+'%' AS chargeback_rate,
    r.total_recovery_active,
    RANK() OVER (ORDER BY r.total_sales DESC) AS sales_rank,
    RANK() OVER (ORDER BY r.total_revenue DESC) AS revenue_rank,
    RANK() OVER (ORDER BY r.cancellation_rate ASC) AS cancellation_rank
FROM Rankings r
ORDER BY
    r.total_sales DESC
    
  ```

In [19]:
Q6.head(100)

,producer_id,total_sales,total_revenue,cancellation_rate,coupon_usage_rate,chargeback_rate,total_recovery_active,sales_rank,revenue_rank,cancellation_rank
0,3,120,6593.180942,"17,5%","40,0%","5,0%",0.0,1,1,1
